Importing Important Libraries

In [63]:
import sagemaker
import boto3
from sagemaker import image_uris
from sagemaker.inputs import TrainingInput
from sagemaker.session import Session

In [64]:
bucket_name = 'applicationbanco2' # nome do bucket criado manualmente
my_region = boto3.session.Session().region_name # pega a regiao

In [65]:
prefix = 'xgboost-as-a-built-in-algo'
output_path = 's3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://applicationbanco2/xgboost-as-a-built-in-algo/output


Downloading The Dataset And Storing in S3

In [66]:
import pandas as pd
import urllib

try:
    urllib.request.urlretrieve("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: download bank_clean.csv.')
except Exception as e:
    print('Data load error: ', e)

try: 
    model_data = pd.read_csv('./bank_clean.csv', index_col=0) # Dataset para verificar se o cliente vai comprar ou não o produto
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: download bank_clean.csv.
Success: Data loaded into dataframe.


In [67]:
model_data.head()

,age,campaign,pdays,previous,no_previous_contact,not_working,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,...,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,y_no,y_yes
0,56,1,999,0,1,0,0,0,0,1,...,0,1,0,0,0,0,1,0,1,0
1,57,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
2,37,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
3,40,1,999,0,1,0,1,0,0,0,...,0,1,0,0,0,0,1,0,1,0
4,56,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0


In [68]:
import numpy as np

train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


In [69]:
# retirndo yes e no dos dados e colocando a coluna do yes como primeira coluna devido ao AWS
import os


pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)


s3 = boto3.resource('s3')
s3.Bucket(bucket_name).upload_file('train.csv', os.path.join(prefix, 'train/train.csv'))

s3_input_train = TrainingInput(
    s3_data=f's3://{bucket_name}/{prefix}/train',
    content_type='csv'
)

In [75]:
# retirndo yes e no dos dados e colocando a coluna do yes como primeira coluna devido ao AWS
import os


pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)


s3 = boto3.resource('s3')
s3.Bucket(bucket_name).upload_file('test.csv', os.path.join(prefix, 'test/test.csv'))

s3_input_test = TrainingInput(
    s3_data=f's3://{bucket_name}/{prefix}/test',
    content_type='csv'
)

Building Models Xgboot- Inbuilt Algorithm

In [71]:
# pegando imagem do container que contem tools para rodas o xgboost
container = image_uris.retrieve(
    framework='xgboost',
    region=boto3.Session().region_name,
    version='1.0-1',
    py_version='py3',
    instance_type='ml.m5.xlarge'
)

In [72]:
#criando hiperparametros
hyperparameters = {
        "max_depth":"5", # profundidade da arvore
        "eta":"0.2", # taxa de aprendizado 
        "gamma":"4",  # ganho minimo para dividisao do nó
        "min_child_weight":"6", # num de ocorrencia minima 
        "subsample":"0.7", # utiizando apenas 70% dos dados na modelo de cada arvore
        "objective":"binary:logistic", # Regressão logistica do tipo binaria
        "num_round":50 # minimo de arvore construido 
        }

In [73]:
# criando estimator para chamar xgboost-contatiner
estimator = sagemaker.estimator.Estimator(
    image_uri = container, # imagme do container Docker usada
    hyperparameters = hyperparameters, # hiperparametros para treinar o modelo
    role = sagemaker.get_execution_role(), # role do usuario para exercutar acessos ao S3
    instance_count=1, # utilizando uma maquina EC2
    instance_type= 'ml.m5.2xlarge', # tipo de EC2 utilizado
    volume_size =5, # 5GB # volumes dos dados que serao armazenados tempo
    output_path=output_path, # caminho do s3
    use_spot_instances=True, # instancias com desconto do s3
    max_run=300, # tempo max de exec 
    max_wait=600,# tempo max de esepera para finalizar o treinamento
)

In [76]:
estimator.fit({'train':s3_input_train, 'validation': s3_input_test})

[05/28/25 14:29:23] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=284658;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=316954;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name:                                       ]8;id=79468;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=869683;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#1042\1042]8;;\
                             sagemaker-xgboost-2025-05-28-14-29-23-163                                             

2025-05-28 14:29:25 Starting - Starting the training job...
2025-05-28 14:29:39 Starting - Preparing the instances for training...
2025-05-28 14:30:15 Downloading - Downloading the training image...
2025-05-28 14:30:51 Training - Training image download completed. Training in progress....
2025-05-28 14:31:22 Uploading - Uploading generated training model[2025-05-28 14:31:13.370 ip-10-2-69-174.ec2.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determ

In [77]:
xgb_predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

[05/28/25 18:54:12] INFO     Creating model with name: sagemaker-xgboost-2025-05-28-18-54-12-013    ]8;id=104801;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=430448;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\

                    INFO     Creating endpoint-config with name                                     ]8;id=823280;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=974179;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#6019\6019]8;;\
                             sagemaker-xgboost-2025-05-28-18-54-12-013                                             

[05/28/25 18:54:13] INFO     Creating endpoint with name sagemaker-xgboost-2025-05-28-18-54-12-013  ]8;id=86634;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=590245;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4841\4841]8;;\

-------!

In [ ]:
from sagemaker.predictor import csv_serializer
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
xgb_predictor.content_type = 'text/csv' # set the data type for an inference
xgb_predictor.serializer = csv_serializer # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

In [ ]:
predictions_array

In [ ]:
m = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))

In [79]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

[05/28/25 19:15:01] WARNING  The endpoint attribute has been renamed in sagemaker>=2.            ]8;id=971872;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py\deprecations.py]8;;\:]8;id=8253;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py#34\34]8;;\
                             See: https://sagemaker.readthedocs.io/en/stable/v2.html for                           
                             details.                                                                              

                    INFO     Deleting endpoint with name: sagemaker-xgboost-2025-05-28-18-54-12-013 ]8;id=560406;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=717291;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4985\4985]8;;\

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)                                  │
│   2 bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)                                  │
│   3 bucket_to_delete.objects.all().delete()                                                      │
│   4                                                                                              │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py:4986 in  │
│ delete_endpoint                                                                                  │
│                                                                                                  │
│   4983 │   │   │   endpoint_name (str): Name of the Amazon SageMaker ``Endpoint`` to delete.     │
│   4984 │   │   """                                                                               │
│   4985 │   │   logger.info("Deleting endpoint with name: %s", endpoint_name)                     │
│ ❱ 4986 │   │   self.sagemaker_client.delete_endpoint(EndpointName=endpoint_name)                 │
│   4987 │                                                                                         │
│   4988 │   def delete_endpoint_config(self, endpoint_config_name):                               │
│   4989 │   │   """Delete an Amazon SageMaker endpoint configuration.                             │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/client.py:570 in     │
│ _api_call                                                                                        │
│                                                                                                  │
│    567 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    568 │   │   │   │   )                                                                         │
│    569 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  570 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    571 │   │                                                                                     │
│    572 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    573                                                                                           │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/context.py:123 in    │
│ wrapper                                                                                          │
│                                                                                                  │
│   120 │   │   │   with start_as_current_context():                                               │
│   121 │   │   │   │   if hook:                                                                   │
│   122 │   │   │   │   │   hook()                                                                 │
│ ❱ 123 │   │   │   │   return func(*args, **kwargs)                                               │
│   124 │   │                                                                                      │
│   125 │   │   return wrapper                                                                     │
│   126                                                      